In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("greatest_roads.ipynb")

### Please restart the kernel after running next cell

In [ ]:
# version shenanigans
!pip install -r requirements.txt --quiet
import otter
grader = otter.Notebook("greatest_roads.ipynb")
assert otter.__version__ >= "4.2.0", "Please restart your kernel."

In [ ]:
import networkx as nx
import tqdm
from heapq import heappush, heappop
import numpy as np
from utils import k_all,n_all,a_all,non_great_roads_all, greatest_roads_all, path_all
import time

# Greatest Roads (Extra Credit)
Here we will implement an algorithm to solve the greatest roads question from this week's problem set. __Note that no debugging help will be given until you have correctly and fully solved the written problem first.__

The following is the prompt from that question 

> Arguably, one of the best things to do in America is to take a great
American road trip. And in America there are some amazing roads to
drive on (think Pacific Coast Highway, Route 66 etc). An intrepid
traveler has chosen to set course across America in search of some
amazing driving. What is the length of the shortest path that hits at
least $k$ of these amazing roads? 

> Assume that the roads in America can be expressed as a directed
weighted graph $G = (V,E,w)$, and that our traveler wishes to drive
across at least $k$ roads from the subset $R \subseteq E$ of ``amazing''
roads. Furthermore, assume that the traveler starts and ends at her
home $a \in V$. You may also assume that the traveler is fine with
repeating roads from $R$, i.e. the $k$ roads chosen from $R$ need not
be unique. 

> Design an algorithm that solves this problem in $O(k(|E|+|V|) \log (k|V|))$ time. 
Provide a 3-part solution. 

Here, implement your solution in the `greatest_roads_solver` function. You may use the following implementation of Dijkstra's in your solution as a subroutine -- in particular, you may find the `get_path` function useful. You may also want to define helper functions to help construct the graph.

*Hints:*  
    1) It may be helpful to define a function which labels vertices in the new graph you create.  
    2) You need to transform the path found on the new graph, back to a path on the original graph.  
__3) All edges are directed. Just because a road is great in one direction, does not mean it's great in both.__

In [ ]:
def shortest_path(graph, v):
    """
    This function assumes that nodes are labelled with integers between 0 and len(graph), where graph 
    is defined below.
    
    args:
        - GRAPH:List[List[Tuple[int,int]]] is an adjacency list representation of the undirected graph.
        g[v] consists of tuples (u, d) such that (v, u) is an edge of weight d.
        - V:int is the start vertex from which we need to find the shortest distances.
        Return:
    returns:
        - DISTANCE:Dict[int,int] a dictionary d such that d[u] is the length of the shortest path
        from V to u. By definition, d[V] = 0.
        - PARENT:Dict[int,int] a dictionary p such that p[u] is the parent of u on the shortest path
        from V to u. In other words, if the shortest path from V to u is (V, x, y, z, u),
        then p[u] = z, p[z] = y, ..., p[x] = V. We define p[V] to be None.
    """
    distance_and_vertex_priority_queue = []
    n = len(graph)
    distance = [float('inf')]*n
    parent = [None]*n
    
    heappush(distance_and_vertex_priority_queue, (0, v))
    distance[v] = 0
        
    while len(distance_and_vertex_priority_queue) > 0:
        distance_to_vertex, vertex = heappop(distance_and_vertex_priority_queue)
        if distance_to_vertex > distance[vertex]:
            continue
        for neighbor, edge_weight in graph[vertex]:
            neighbor_distance = distance[neighbor]
            new_neighbor_distance = distance_to_vertex + edge_weight
            
            if new_neighbor_distance < neighbor_distance:
                distance[neighbor] = new_neighbor_distance
                parent[neighbor] = vertex
                heappush(distance_and_vertex_priority_queue, (new_neighbor_distance, neighbor))
                
    return distance, parent

def get_path(w, parent):
    """
    Take in a vertex W and a dictionary PARENT, as described in the docstring
    for the shortest_path method, and return the path from V to W, where V
    is the vertex from which we ran Dijkstra's algorithm.
    """
    path = []
    while w is not None:
        path.append(w)
        w = parent[w]
    path.reverse()
    return path

def get_distance(w, distance):
    """
    Take in a vertex W and a dictionary DISTANCE, as described in the docstring
    for the shortest_path method, and return the length of the shortest path from V to W,
    where V is the vertex from which we ran Dijkstra's algorithm.
    """
    return distance[w]

In [ ]:
...

def greatest_roads_solver(non_great_roads, greatest_roads, k, a, n):
    """
    Returns the shortest path which starts at node a and ends at node a which goes through k amazing roads
    
    args:
        non_great_roads:List[Tuple[int,int]] = a list of tuples containing all roads which are not amazing roads.
            The tuple (u,v) represents a directed edge corresponding to a non-amazing road.
        greatest_roads:List[Tuple[int,int]] = a list of tuples containing all roads which are amazing roads.
            The tuple (u,v) represents a directed edge corresponding to a non-amazing road
        k:int = the number of amazing roads the path must traverse
        a:int = the node representing home, which the path must start and end at
        n:int = the number of nodes in the graph
    returns:
        a List[int] corresponding to the shortest path passing through k amazing roads. For example, if that 
            path is s->a->b->c->s, return the list [s,a,b,c,s]
    """
    ...


In [ ]:
grader.check("q1")

### Debugging
The otter tests are pasted here for your convenience. Feel free to add whatever print statements or assertions you'd like when debugging.

In [ ]:
def is_great(u,v, greatest_roads):
    for a,b,_ in greatest_roads:
        if a == u and b == v:
            return True
    return False
for k,n,a,non_great_roads, greatest_roads, sol in tqdm.tqdm(zip(k_all,n_all,a_all,non_great_roads_all, greatest_roads_all, path_all),total=len(k_all)):
    non_great_roads = [(edge[0],edge[1],edge[2]) for edge in non_great_roads]
    greatest_roads = [(edge[0],edge[1],edge[2]) for edge in greatest_roads]

    # create adjacency list from the edge lists
    adj_list = [[-1 for j in range((k+1)*n)] for i in range((k+1)*n)]
    for u,v,w in non_great_roads:
        adj_list[u][v] = w
    for u,v,w in greatest_roads:
        adj_list[u][v] = w
    path = greatest_roads_solver(non_great_roads, greatest_roads, k, a, n)

    # check the path only contains edges in the graph AND that is passes through k amazing roads
    num_greatest_roads = 0
    for i in range(len(path)-1):
        assert adj_list[u][v] != -1, f"Your path includes an edge that's not in the graph!"
        if is_great(path[i], path[i + 1], greatest_roads):
            num_greatest_roads += 1
    assert num_greatest_roads >= k, f"Your path contains fewer than k amazing roads!"

    # checks that the path starts and ends at the right nodes
    assert path[0] == sol[0], f"Your path does not start at the correct node."
    assert path[len(path) - 1] == sol[len(sol) - 1], f"Your path does not end at the correct node."

    # checks that the path is the shortest path
    distance1 = np.sum([adj_list[path[i]][path[i+1]] for i in range(len(path) - 1)])
    distance2 = np.sum([adj_list[sol[i]][sol[i+1]] for i in range(len(sol) - 1)])
    assert distance1 == distance2, f"Your path is not the possible shortest path."

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit.

In [ ]:
grader.export(pdf=False, force_save=True, run_tests=True)